In [6]:
# pip install langchain_openai langchain_core langgraph python-dotenv openai
# pip install -U langgraph "langchain[openai]"
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

from openai import OpenAI
from dotenv import load_dotenv
import os
# 1
from langsmith import traceable


load_dotenv(override=True, dotenv_path="../.env.local")
my_api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI(api_key=my_api_key)


# --- Shared State ---
class LibraryState(TypedDict):
    question: str
    faq_answer: str
    checkout_info: str
    final_answer: str
#2
@traceable(name="ClassifierAgent")
def ClassifierAgent(state: LibraryState):
    print("ClassifierAgent ran")
    print(f"state: {state}")

    # Build the LLM messages
    message_to_llm = [
        {"role": "system", "content": '''You are a classifier agent in a library system. Decide if the user is asking about book availability/checkout or about library FAQs. 
        Reply with JSON containing keys: faq_answer and checkout_info.'''},
        {"role": "user", "content": f"Question: {state['question']}"}
    ]

    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=message_to_llm,
        temperature=0.2,   # keep it deterministic for classification
        max_tokens=150,
    )

    print (response)
    # Extract the content from the response
    answer = response.choices[0].message.content

    # Ideally, parse as JSON — here assuming model returns a dict-like string
    try:
        import json
        parsed = json.loads(answer)
        return {
            "faq_answer": parsed.get("faq_answer", ""),
            "checkout_info": parsed.get("checkout_info", "")
        }
    except Exception:
        # fallback if LLM gives plain text
        return {"faq_answer": answer, "checkout_info": ""}


@traceable(name="FAQAgent")
def FAQAgent(state: LibraryState):
    print("FAQAgent ran")
    print(f"FAQAgent state", state)
    if not state.get("faq_answer"):
        return {"faq_answer": "Default FAQ: Library rules apply"}
    return {}

@traceable(name="CheckoutAgent")
def CheckoutAgent(state: LibraryState):
    print("CheckoutAgent ran")
    if not state.get("checkout_info"):
        return {"checkout_info": "Checkout info: Not requested"}
    return {}

@traceable(name="ResponseAgent")
def ResponseAgent(state: LibraryState):
    print("ResponseAgent ran")
    # final = f"Q: {state['question']}\n"
    # if state.get("faq_answer"):
    #     final += f"FAQ: {state['faq_answer']}\n"
    # if state.get("checkout_info"):
    #     final += f"Checkout: {state['checkout_info']}"


    # Build the LLM messages
    message_to_llm = [
        {"role": "system", "content": '''You are generating a human friendly response  for a library system based on given data 
         You will get FAQ_answer and Checkout_info - combine them to give a meningful answer'''},
        {"role": "user", "content": f"Question: {state['question']}, faq_answer: {state['faq_answer']}, checkout_info: {state['checkout_info']}"}
    ]

    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=message_to_llm,
        temperature=0.2,   # keep it deterministic for classification
        max_tokens=150,
    )

    print (response)
    # Extract the content from the response
    answer = response.choices[0].message.content
    return {"final_answer": answer}


# --- Build the Graph ---
builder = StateGraph(LibraryState)
builder.add_node("ClassifierAgent", ClassifierAgent)
builder.add_node("FAQAgent", FAQAgent)
builder.add_node("CheckoutAgent", CheckoutAgent)
builder.add_node("ResponseAgent", ResponseAgent)

builder.add_edge(START, "ClassifierAgent")
builder.add_edge("ClassifierAgent", "FAQAgent")
builder.add_edge("ClassifierAgent", "CheckoutAgent")
builder.add_edge("FAQAgent", "ResponseAgent")
builder.add_edge("CheckoutAgent", "ResponseAgent")
builder.add_edge("ResponseAgent", END)

graph = builder.compile()



In [7]:


# --- Visualize ---
print(graph.get_graph().draw_ascii())


               +-----------+              
               | __start__ |              
               +-----------+              
                     *                    
                     *                    
                     *                    
            +-----------------+           
            | ClassifierAgent |           
            +-----------------+           
              ***          **             
             *               **           
           **                  **         
+---------------+           +----------+  
| CheckoutAgent |           | FAQAgent |  
+---------------+           +----------+  
              ***          **             
                 *       **               
                  **   **                 
             +---------------+            
             | ResponseAgent |            
             +---------------+            
                     *                    
                     *                    
           

In [8]:
# --- Run ---
result = graph.invoke({"question": "Is The new best seller available? When does library open?"})
print("\n--- Final Answer ---")
print(result["final_answer"])


ClassifierAgent ran
state: {'question': 'Is The new best seller available? When does library open?'}
ChatCompletion(id='chatcmpl-D8IdKz0LuqFx0rgLkxEZaZm3hTuAf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "faq_answer": "The library opens at 9 AM and closes at 9 PM from Monday to Saturday, and from 12 PM to 6 PM on Sundays. For specific book availability, please check our online catalog or contact the library directly.",\n  "checkout_info": "The new best seller\'s availability can be checked through our online catalog or at the library\'s circulation desk."\n}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1770870622, model='gpt-4.1-nano-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_de604bd877', usage=CompletionUsage(completion_tokens=81, prompt_tokens=65, total_tokens=146, completion_tokens_details=CompletionTokensDetails(a

In [5]:
# result = graph.invoke({"question": "Is The Hobbit available?"})
# print("\n--- Final Answer ---")
# print(result["final_answer"])
